In [1]:
import pandas as pd
import numpy as np
import ast
import re

In [3]:
premanual_processed_df = pd.read_excel('D:/Only_GDB_Operation/Grant Database Final Pipeline/SAMPLE_Merged_File_Premanual_for_main_file.xlsx')

In [4]:
gdb_df = pd.read_excel('D:/Only_GDB_Operation/FInal trained data/Updated_GDB_Training_data.xlsx')

In [5]:
premanual_processed_df.columns

Index(['IGR Village', 'SRO Name', 'Document No', 'Transaction Type',
       'Agreement Price(INR)', 'Bajarbhav', 'Bhumapan', 'Transaction Date',
       'Google_Translation_Proj_Name', 'Floor No', 'Unit No', 'Property Type',
       'carpet_sqmt', 'balcony_sqmt', 'terrace_sqmt',
       'Net Carpet Area sqmt (KHESTRAFAL)', 'Seller Name', 'Purchaser Name',
       'Rate/sqft(INR)'],
      dtype='object')

In [6]:
# premanual_df.rename(columns={'date': 'Transaction Date', 'unit_clean': 'Unit No', 'floor_clean': 'Floor No', 
#                             'mobadla': 'Agreement Price(INR)', 'Kshetrafal': 'Net Carpet Area sqmt (KHESTRAFAL)', 'doc_type': 'Transaction Type', 'bhumapan': 'Bhumapan',
#                             'Property_Type': 'Property Type','village_Name_Eng': 'IGR Village', 'sro_name': 'SRO Name', 'doc_num': 'Document No', 'bajarbhav': 'Bajarbhav'}, inplace=True)

In [6]:
gdb_df

,Index,Project Name,IGR Village,Rera Location,Developer Occurrences,Survey Extracted Info Occurrences,Actual Data,Maximum_Developer_Occurrences,Maximum_Survey_No_Occurences
0,104,1 Goldleaf,Hingane Budruk (Karve Nagar),Hingane Budruk,{'Unknown Developer': 110},{'14/4बी/2/2/एसर्': 110},110.0,[],['14/4बी/2/2/एसर्']
1,105,1 Hallmark Aveneu,Ravet,Ravet,"{'अमित गजानन विलायतकर 45 प्लॉट': 1, 'अर्चन शंक...","{'91 हिस्सा नंबर ': 31, '91/2अ व स.नं. 9': 1, ...",46.0,"['डॉ. मृणाल नितीन फोंडेकर 40', 'मे. हॉलमार्क अ...",['91 हिस्सा नंबर ']
2,109,10 Biz Park,Lohegaon,Lohegaon,{'किर्तीकांत जसवंतराय वगाडीआ वय58 पत्ताप्लॉट':...,"{'227 व 227/1 ते ': 147, '227227/1 ते 227': 17...",168.0,"['दिया विजय अडवाणी 38 प्लॉट', 'नीरज नरेंद्र व्...",['227 व 227/1 ते ']
3,110,10 Elite,Pimple Gurav,Pimple Gurav,"{'गणेश अशोक गायकवाड 42 प्लॉट': 2, 'गिरीश विठ्ठ...","{'39/1/139/2/139/': 6, '39(पै)40(पै)व 4': 10, ...",19.0,"['गणेश अशोक गायकवाड 42 प्लॉट', 'वृषाली विजय दे...",['39(पै)40(पै)व 4']
4,117,10 Heaven,Vadgaon,Vadgaon,"{'युगंधरा आनंद गायकवाड व प्रविण': 1, 'वास्तुश्...","{'106/2 मधील मंजू': 2, '106 मधील मंजूर ': 8}",10.0,['वास्तुश्री इंन्फ्रा तर्फे भागीदार प्रविण'],['106 मधील मंजूर ']
...,...,...,...,...,...,...,...,...,...
12384,9581,Westworld,NaN,Balewadi,NaN,NaN,NaN,[],[]
12385,9582,Woodshire,NaN,Gurgaon,NaN,NaN,NaN,[],[]
12386,9583,Wyng,NaN,Somwar Peth,NaN,NaN,NaN,[],[]
12387,9584,Yashvi Green,NaN,-,NaN,NaN,NaN,[],[]


In [7]:
# Convert the string representation to a list
gdb_df['Maximum_Developer_Occurrences'] = gdb_df['Maximum_Developer_Occurrences'].apply(ast.literal_eval)
gdb_df['Maximum_Survey_No_Occurences'] = gdb_df['Maximum_Survey_No_Occurences'].apply(ast.literal_eval)

# Ensuring the Index column is treated as string to avoid float conversion
gdb_df['Index'] = gdb_df['Index'].astype(str)

In [9]:
# Define the test village
test_village = "Hadapsar"  # Replace with the actual name of the test village

# Create an empty set to store processed villages
processed_villages = set()

# Iterate through each row in gdb_df
for index, row in gdb_df.iterrows():
    # Get the village name
    village = row['IGR Village']
    
    # Check if the village has already been processed
    if village not in processed_villages:
        # Add the village to the set of processed villages
        processed_villages.add(village)
        
        # Only process the test village
        if village == test_village:
            # Filter premanual_df based on village name
            filtered_df = premanual_df[premanual_df['IGR Village'] == village].copy()
            
            print(f"Processing village '{village}':")
            print('Entering the loop now')
            
            # Iterate through each record in filtered_df
            for idx, r in filtered_df.iterrows():
                # Check if Seller Name and Bhumapan are not NaN
                if not pd.isnull(r['Seller Name']) and not pd.isnull(r['Bhumapan']):
                    match_found = False  # Initialize match_found as False
                    
                    # Iterate through each row in gdb_df again to check all project names
                    for i, project_row in gdb_df.iterrows():
                        if project_row['IGR Village'] == village:
                            match_type = None  # Initialize match type as None
                            
                            # Check if any string in Maximum_Developer_Occurrences column is part of Seller Name
                            dev_match = False
                            for dev_occurrence in project_row['Maximum_Developer_Occurrences']:
                                if dev_occurrence in r['Seller Name']:
                                    dev_match = True
                                    break  # Break the loop if a match is found
                            
                            # Check if any string in Maximum_Survey_No_Occurrences column is part of Bhumapan
                            survey_match = False
                            for survey_occurrence in project_row['Maximum_Survey_No_Occurences']:
                                if survey_occurrence in r['Bhumapan']:
                                    survey_match = True
                                    break  # Break the loop if a match is found
                            
                            # Determine match type based on dev_match and survey_match
                            if dev_match and survey_match:
                                match_type = 'Both'
                            elif dev_match:
                                match_type = 'Developer Occurrence'
                            elif survey_match:
                                match_type = 'Survey Number'
                            
                            # Assign the project name and match type to the record in filtered_df
                            if match_type:
                                filtered_df.at[idx, 'Project Name'] = project_row['Project Name']
                                filtered_df.at[idx, 'Match Type'] = match_type
                                filtered_df.at[idx, 'Index'] = project_row['Index'].astype('int64')
                                match_found = True  # Set match_found to True if any match is found
                                print(f"Match found for village '{village}': {match_type}")
                                
                                
                    # If no match is found, set the Match Type to 'No Match'
                    if not match_found:
                        filtered_df.at[idx, 'Match Type'] = 'No Match'

            # # Update premanual_df with the modified filtered_df2
            # premanual_df.update(filtered_df2)
            
            # Break the loop after processing the test village
            break

Processing village 'Hadapsar':
Entering the loop now
Match found for village 'Hadapsar': Both
Match found for village 'Hadapsar': Both
Match found for village 'Hadapsar': Both
Match found for village 'Hadapsar': Both
Match found for village 'Hadapsar': Both
Match found for village 'Hadapsar': Both
Match found for village 'Hadapsar': Both
Match found for village 'Hadapsar': Both
Match found for village 'Hadapsar': Both
Match found for village 'Hadapsar': Both
Match found for village 'Hadapsar': Both
Match found for village 'Hadapsar': Both
Match found for village 'Hadapsar': Both
Match found for village 'Hadapsar': Both
Match found for village 'Hadapsar': Both
Match found for village 'Hadapsar': Both
Match found for village 'Hadapsar': Both
Match found for village 'Hadapsar': Both
Match found for village 'Hadapsar': Survey Number


C:\Users\Admin\AppData\Local\Temp\ipykernel_4740\863568692.py:139: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'A104' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  filtered_df.at[idx, 'Index'] = project_row['Index']


Match found for village 'Hadapsar': Survey Number
Match found for village 'Hadapsar': Survey Number
Match found for village 'Hadapsar': Both
Match found for village 'Hadapsar': Both
Match found for village 'Hadapsar': Both
Match found for village 'Hadapsar': Both
Match found for village 'Hadapsar': Both
Match found for village 'Hadapsar': Both


# Input the list of villages

In [8]:
# give the list of Villages
test_villages = ["Holewadi", "Chandoli"]  # Replace with the actual names of the test villages

# Create an empty set to store processed villages
processed_villages = set()

# Define the desired order of columns
desired_order = ['Index', 'Project Name', 'Google_Translation_Proj_Name', 'Transaction Date', 'Unit No', 'Property Type',
                 'Floor No', 'Net Carpet Area sqmt (KHESTRAFAL)', 'carpet_sqmt','balcony_sqmt', 'terrace_sqmt', 'Agreement Price(INR)', 
                 'Rate/sqft(INR)','Transaction Type', 'Bhumapan', 'Rera Location','IGR Village','SRO Name', 'Document No', 'Bajarbhav', 
                 'Seller Name', 'Purchaser Name', 'Match Type']

# Iterate through each test village
for test_village in test_villages:
    # Check if the village has already been processed
    if test_village not in processed_villages:
        # Add the village to the set of processed villages
        processed_villages.add(test_village)
        
        # Filter premanual_processed_df based on village name
        filtered_df = premanual_processed_df[premanual_processed_df['IGR Village'] == test_village].copy()
        
        print(f"Processing village '{test_village}':")
        print('Entering the loop now')
        
        # Iterate through each record in filtered_df
        for idx, r in filtered_df.iterrows():
            # Check if Seller Name and Bhumapan are not NaN
            if not pd.isnull(r['Seller Name']) and not pd.isnull(r['Bhumapan']):
                match_found = False  # Initialize match_found as False
                
                # Iterate through each row in gdb_df again to check all project names
                for i, project_row in gdb_df.iterrows():
                    if project_row['IGR Village'] == test_village:
                        match_type = None  # Initialize match type as None
                        
                        # Check if any string in Maximum_Developer_Occurrences column is part of Seller Name
                        dev_match = False
                        for dev_occurrence in project_row['Maximum_Developer_Occurrences']:
                            if dev_occurrence in r['Seller Name']:
                                dev_match = True
                                break  # Break the loop if a match is found
                        
                        # Check if any string in Maximum_Survey_No_Occurrences column is part of Bhumapan
                        survey_match = False
                        for survey_occurrence in project_row['Maximum_Survey_No_Occurences']:
                            if survey_occurrence in r['Bhumapan']:
                                survey_match = True
                                break  # Break the loop if a match is found
                        
                        # Determine match type based on dev_match and survey_match
                        if dev_match and survey_match:
                            match_type = 'Both'
                        elif dev_match:
                            match_type = 'Developer Occurrence'
                        elif survey_match:
                            match_type = 'Survey Number'
                        
                        # Assign the project name and match type to the record in filtered_df
                        if match_type:
                            filtered_df.at[idx, 'Project Name'] = project_row['Project Name']
                            filtered_df.at[idx, 'Match Type'] = match_type
                            filtered_df.at[idx, 'Index'] = project_row['Index']
                            filtered_df.at[idx, 'Rera Location'] = project_row['Rera Location']
                            match_found = True  # Set match_found to True if any match is found
                            print(f"Match found for village '{test_village}': {match_type}")
                            # No break here, continue to check all project names for the village
                        
                # If no match is found, set the Match Type to 'No Match'
                if not match_found:
                    filtered_df.at[idx, 'Match Type'] = 'No Match'

                # Ensure all necessary columns are present in filtered_df
        for column in desired_order:
            if column not in filtered_df.columns:
                filtered_df[column] = None

        # Reorder the columns in filtered_df
        filtered_df = filtered_df[desired_order]

        # Reorder the columns in filtered_df
        filtered_df = filtered_df[desired_order]

        # Save the filtered_df to an Excel file named after the village
        output_file = f"{test_village}_processed.xlsx"
        filtered_df.to_excel(output_file, index=False)
        print(f"Saved processed data for village '{test_village}' to '{output_file}'")

Processing village 'Hadapsar':
Entering the loop now
Match found for village 'Hadapsar': Survey Number
Match found for village 'Hadapsar': Survey Number
Match found for village 'Hadapsar': Survey Number


C:\Users\Admin\AppData\Local\Temp\ipykernel_14556\2749643504.py:64: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Hadapsar' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  filtered_df.at[idx, 'Rera Location'] = project_row['Rera Location']


Match found for village 'Hadapsar': Developer Occurrence
Match found for village 'Hadapsar': Developer Occurrence
Match found for village 'Hadapsar': Developer Occurrence
Match found for village 'Hadapsar': Developer Occurrence
Match found for village 'Hadapsar': Developer Occurrence
Match found for village 'Hadapsar': Developer Occurrence
Match found for village 'Hadapsar': Developer Occurrence
Match found for village 'Hadapsar': Developer Occurrence
Match found for village 'Hadapsar': Developer Occurrence
Match found for village 'Hadapsar': Developer Occurrence
Match found for village 'Hadapsar': Developer Occurrence
Match found for village 'Hadapsar': Developer Occurrence
Match found for village 'Hadapsar': Developer Occurrence
Match found for village 'Hadapsar': Developer Occurrence
Match found for village 'Hadapsar': Developer Occurrence
Match found for village 'Hadapsar': Developer Occurrence
Match found for village 'Hadapsar': Developer Occurrence
Match found for village 'Hadaps

In [10]:
filtered_df

,IGR Village,SRO Name,Document No,Transaction Type,Agreement Price(INR),Bajarbhav,Bhumapan,Transaction Date,Google_Translation_Proj_Name,Floor No,...,carpet_sqmt,balcony_sqmt,terrace_sqmt,Net Carpet Area sqmt (KHESTRAFAL),Seller Name,Purchaser Name,Rate/sqft(INR),Match Type,Project Name,Index
0,Hadapsar,सह दु.नि. हवेली 15,3639,Sale Agreement,4367000,0.000,", इतर माहिती: , इतर माहिती: गांव मौजे हडपसर ये...",25/02/2020,Magarpatta City,2.0,...,41.06,NaN,NaN,41.0600,1) पंजाब नॅशनल बँक तर्फे चीफ मॅनेजर सुधीर कुम...,"2) सुनिल बबनराव सोनवणे -- -- 31 प्लॉट नं: --, ...",9880.764715,No Match,NaN,NaN
1,Hadapsar,सह दु.नि. हवेली 15,7318,Sale Agreement,7830000,6116733.000,", इतर माहिती: विभाग क्र. 27.4 दर- 57,570/- गाव...",31/07/2020,Prithvi Presidio or Tower no. in B,6.0,...,76.25,NaN,NaN,76.2500,1) मान्यता देणार अभय अभय पंढरीनाथ केले तर्फे ...,"2) सुमित नरेंद्र दहाड - - 29 प्लॉट नं: -, माळा...",9539.996710,Both,Prithvi Proximus,5538.0
2,Hadapsar,सह दु.नि. हवेली 15,8182,Sale Agreement,4000000,3879968.000,", इतर माहिती: , इतर माहिती: पुणे मनपा हद्दीतील...",19/08/2020,Isha Empire Cooperative Housing Society Limited,8.0,...,NaN,NaN,NaN,78.9700,"1) श्री.रज्जाक अहमद पठाण 41 प्लॉट नं: ., माळा...","2) सौ. रेणुका मनोज साकळे 46 प्लॉट नं: ., माळा ...",4705.699218,No Match,NaN,NaN
3,Hadapsar,सह दु.नि. हवेली 15,9988,Sale deed,4200000,3922377.600,", इतर माहिती: , इतर माहिती: (विभाग क्र.30/464 ...",27/09/2020,Aayan Ashoknagar Apartments or Yojana Building...,6.0,...,NaN,NaN,NaN,81.4100,1) लिहून देणार - श्री.शंतनु प्रकाश महाजन . 4...,2) लिहून घेणार - सौ.सोनाली विवेक महाजन . 45 प...,4792.894247,No Match,NaN,NaN
4,Hadapsar,सह दु.नि. हवेली 15,9990,Transfer Deed,6000000,0.000,", इतर माहिती: गाव मौजे हडपसर येथील सर्व्हे नं....",27/09/2020,DRC NO. 004572 or in,NaN,...,NaN,NaN,NaN,510.9600,1) मे. नाईकनवरे डेव्हलपर्स प्रायवेट लिमिटेड त...,1) श्री. अमोल अजित रावेतकर तर्फे नोंदणी साठी व...,1090.914359,No Match,NaN,NaN
5,Hadapsar,सह दु.नि. हवेली 15,10755,Transfer Deed,4000000,3879968.000,", इतर माहिती: , इतर माहिती: पुणे मनपा हद्दीतील...",09/10/2020,Isha Empire Cooperative Housing Society Limited,8.0,...,NaN,NaN,NaN,78.9700,"1) श्री.रज्जाक अहमद पठाण - - 41 प्लॉट नं: -, ...","2) सौ. रेणुका मनोज साकळे - - 46 प्लॉट नं: -, म...",4705.699218,No Match,NaN,NaN
6,Hadapsar,सह दु.नि. हवेली 15,11095,Transfer Deed,1897155,0.000,", इतर माहिती: गाव मौजे हडपसर(रामटेकडी)येथील सर...",15/10/2020,NaN,NaN,...,NaN,NaN,NaN,150.0000,1) आदिनाथ लॅण्डस्पेस (एल एल पी) तर्फे भागीदार...,1) अतुल बिल्डर्स तर्फे भागीदार अशोक धनराज चोरड...,1175.000000,No Match,NaN,NaN
7,Hadapsar,सह दु.नि. हवेली 15,12457,Sale Agreement,3210000,3553714.255,", इतर माहिती: , इतर माहिती: (विभाग क्र.30/466 ...",04/11/2020,Selen Park Co-op. Housing Society Ltd. in buil...,9.0,...,58.92,NaN,NaN,58.9200,"1) श्री.संतोष पांडे . 39 प्लॉट नं: ., माळा न...","2) श्री.सचिन जगरिया . 29 प्लॉट नं: ., माळा नं...",5061.376043,No Match,NaN,NaN
8,Hadapsar,सह दु.नि. हवेली 15,13458,Sale Agreement,2855100,3622147.200,", इतर माहिती: , इतर माहिती: गांव मौजे हडपसर ये...",21/11/2020,Building B or Building B in Hillside Project,7.0,...,NaN,NaN,NaN,63.9200,1) बेल वेदर डेव्हलपर्स प्रायव्हेट लिमिटेड तर्...,"2) संदेश काकासाहेब भुजबळ - - 34 प्लॉट नं: -, म...",4149.644274,No Match,NaN,NaN
9,Hadapsar,सह दु.नि. हवेली 15,15203,Assignment Deed,4775000,4203442.950,", इतर माहिती: , इतर माहिती: (विभाग क्र.30/466 ...",14/12/2020,Aayan Nilesh Plaza Co-op. Housing Society Ltd....,1.0,...,61.50,NaN,NaN,81.7800,1) श्री.मधुकर निवृत्ती न्हावले यांच्या तर्फे ...,"1) श्री.अफराज निसार अत्तार . 24 प्लॉट नं: ., ...",5424.410908,No Match,NaN,NaN


In [ ]:
premanual_df2.to_excel('Hinjewadi_allyears_with_project_names.xlsx', index=False)

In [ ]:
filtered_df.to_excel('SAMPLE2.xlsx', index=False)